In [1]:
import pandas as pd
from model.py import model_response
from main.py import user_stories_db, retrieve_user_stories, metadata_retrieval, match_mongo, main
from get_schema.py import data_recursion, extract_keys_bfs, single_sub_keys
from feature_extraction.py import obtain_nested_data, using_zero_shot, using_rouge_similarity, main_feature

client = OpenAI(base_url = "http://localhost:8000/v1", api_key = "token")

# Function to handle file-based querying and store results in a CSV
def query_and_store_results(input_file, output_csv, database_name, collection_name, milvus_collection_name):
    results = []

    with open(input_file, 'r') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        query = line

        # Query the RAG model
        mongo_user_story_ids = retrieve_user_stories(query, milvus_collection_name, 0.2)
        mongo_user_story_ids = list(set(mongo_user_story_ids[:5]))  # Take first 5 unique IDs
        chunk_data = metadata_retrieval(database_name, collection_name, query, mongo_user_story_ids)
        response = model_response(query, "\n".join(chunk_data))
        results.append({'Query': query, 'Response': response})

        # Check if the next line is empty to determine "T" or "F" input
        if i + 1 < len(lines) and lines[i + 1].strip() != "":
            response_t = model_response("T", "\n".join(chunk_data))
        else:
            response_f = model_response("F", "\n".join(chunk_data))
            break

    # Convert results to DataFrame and save to CSV
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")

ModuleNotFoundError: No module named 'model'

First working tester

In [ ]:
import pandas as pd
from model import model_response
from main import user_stories_db, retrieve_user_stories, metadata_retrieval, main
from get_schema import data_recursion, single_sub_keys
from feature_extraction import obtain_nested_data, using_zero_shot, using_rouge_similarity, main_feature

database_name = "OrgqrS1HZ"
collection_name = "userStories"
milvus_collection_name = "temp" 

# Function to handle file-based querying and store results in a CSV
def query_and_store_results(input_file, output_csv, database_name, collection_name, milvus_collection_name):
    results = []

    # Open the input file
    with open(input_file, 'r') as file:
        lines = file.readlines()

    # Process each query line
    for i, line in enumerate(lines):
        query = line.strip()  # Strip extra spaces or newline

        # Query the RAG model
        mongo_user_story_ids = retrieve_user_stories(query, milvus_collection_name, 0.2)
        mongo_user_story_ids = list(set(mongo_user_story_ids[:5]))  # Take first 5 unique IDs

        # Get chunked data from metadata
        chunk_data = metadata_retrieval(database_name, collection_name, query, mongo_user_story_ids)
        response = model_response(query, "\n".join(chunk_data))
        results.append({'Query': query, 'Response': response})

        # Check continuation
        if i + 1 < len(lines) and lines[i + 1].strip().upper() == "T":
            continue  # Continue to the next query
        elif i + 1 >= len(lines) or lines[i + 1].strip().upper() == "F":
            break  # Stop if next line is empty or 'F'

    # Convert results to DataFrame and save to CSV
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")

if __name__ == "__main__":
    query_and_store_results("input.txt", "output.csv", database_name, collection_name, milvus_collection_name)

Code for auto rouge scores

In [ ]:
import pandas as pd

# Load the original CSV and the new CSV with the column to add
original_df = pd.read_csv('original_data.csv')
new_df = pd.read_csv('new_data.csv')

# Check if both DataFrames have the same length (if they are aligned)
if len(original_df) == len(new_df):
    # Add the new column to the original DataFrame
    original_df['NewColumn'] = new_df['NewColumn']

    # Save the updated DataFrame back to CSV
    original_df.to_csv('updated_original_data.csv', index=False)
    print("Column added and saved to 'updated_original_data.csv'")
else:
    print("DataFrames do not have the same length. Cannot directly add the column.")


Code to access mongoDB

ServerSelectionTimeoutError: localhost:27018: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 673476ca939d077803ff3e48, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27018) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27018: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>